In [6]:
import sys
import os
from pathlib import Path

sys.path.append(str(Path(os.getcwd()).parent))
from pyparadigm.paradigm import LcsSearcher

In [7]:
infile = "/home/alexeysorokin/data/neural_tagging/data/low-resource/evk-sample.train.ud"

In [8]:
from neural_LM.UD_preparation.extract_tags_from_UD import read_tags_infile

In [13]:
tag_sents, source_sents, lemma_sents = read_tags_infile(infile, to_lower=True, return_source_words=True, return_lemmas=True)

/home/alexeysorokin/data/neural_tagging/data/low-resource/evk-sample.train.ud


In [15]:
tag_sents[0], source_sents[0], lemma_sents[0]

(['INTJ',
  'ADV',
  'PART',
  'VERB,Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin',
  'PRON,Case=Nom|Number=Sing|Person=1|PronType=Prs',
  'NOUN,Case=Dat|Number=Sing',
  'INTJ'],
 ['nu', 'idu', 'ka', 'baldit͡ʃaːw', 'bi', 'mohaduː', 'aha'],
 ['nu', 'idu', 'ka', 'baldi', 'bi', 'moha', 'aha'])

In [23]:
from collections import defaultdict

changed_counts = defaultdict(int)
total_counts = defaultdict(int)
for elem in zip(tag_sents, source_sents, lemma_sents):
    for i, (tag, word, lemma) in enumerate(zip(*elem)):
        pos = tag.split(",")[0]
        total_counts[pos] += 1
        if word.lower() != lemma.lower():
            changed_counts[pos] += 1
print(changed_counts)
print(total_counts)

defaultdict(<class 'int'>, {'VERB': 4172, 'NOUN': 2607, 'DET': 218, 'PRON': 594, 'PROPN': 201, 'ADV': 95, 'ADJ': 146, 'INTJ': 3, 'X': 24, 'PART': 3, 'NUM': 5})
defaultdict(<class 'int'>, {'INTJ': 464, 'ADV': 1388, 'PART': 632, 'VERB': 4243, 'PRON': 1204, 'NOUN': 3494, 'ADJ': 530, 'X': 1866, 'DET': 706, 'NUM': 137, 'CCONJ': 186, 'PROPN': 548, 'SCONJ': 71})


In [39]:
word_pairs, tags = [], []
for elem in zip(tag_sents, source_sents, lemma_sents):
    for i, (tag, word, lemma) in enumerate(zip(*elem)):
        splitted = tag.split(",", maxsplit=1)
        pos = splitted[0]
        if pos in ["VERB", "NOUN", "PROPN"]:
            word_pairs.append((lemma, word))
            feats = splitted[1] if len(splitted) >= 2 else pos
            tags.append((pos, feats))
len(word_pairs)            

8285

In [40]:
lcs_searcher = LcsSearcher(method="modified_Hulden")

In [41]:
paradigms = lcs_searcher.calculate_paradigms(word_pairs, count_paradigms=True)

In [42]:
paradigm_counts_by_tags = defaultdict(lambda: defaultdict(set))
for (paradigm, fragments), (lemma, word), (pos, feats) in zip(paradigms, word_pairs, tags):
    if pos == "PROPN":
        pos = "NOUN"
    paradigm = "#".join(paradigm)
    paradigm_counts_by_tags[",".join([pos, feats])][paradigm].add(lemma)    

In [45]:
with open("../neural_tagging/results/low-resource/evenk-paradigm-stats.out", "w", encoding="utf8") as fout:
    for descr, descr_data in sorted(paradigm_counts_by_tags.items()):
        fout.write("{}\t{}\n".format(descr, sum(len(x[1]) for x in descr_data.items())))
        for pattern, lemmas in sorted(descr_data.items(), key=(lambda x:-len(x[1]))):
            fout.write("{}\t{}\t{}\n".format(pattern, len(lemmas), ",".join(lemmas)))
        fout.write("\n")